<a href="https://colab.research.google.com/github/Tristan-Brown1096/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

## Practice 1-Sample T-tests

One Sample t-tests determine whether or not a sample mean is statistically different from some known (or hypothesized) population mean. 

### 1) Load the Data
- Use the [automobile dataset](https://archive.ics.uci.edu/ml/datasets/Automobile)
- Fix the column headers
- Make sure NaNs are used to indicate missing values

Feel free to add code cells and text cells as needed throughout the assignment.

In [1]:
### YOUR WORK HERE
# importing pandas, numpy, and stats
import pandas as pd
import numpy as np
from scipy import stats

In [8]:
#downloading cars dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
!head imports-85.data

--2020-07-13 19:41:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25936 (25K) [application/x-httpd-php]
Saving to: ‘imports-85.data.1’

imports-85.data.1   100%[===================>]  25.33K  --.-KB/s    in 0.04s   

2020-07-13 19:41:40 (683 KB/s) - ‘imports-85.data.1’ saved [25936/25936]

3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,171.20,65.50,52.40,2823,ohcv,six,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
2,164,audi,gas,std,four,sedan,fwd,front,99.80,176.60,

In [15]:
#creating cars dataframe
cars_columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'number-of-doors',
                'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 
                'height', 'curb-weight', 'engine-type','number-of-cylinders', 'engine-size', 
                'fuel-system', 'bore', 'stroke', 'compression', 'horespower', 'peak-rpm', 'city-mpg',
                'highway-mpg', 'price']
cars = pd.read_csv('imports-85.data', na_values='?', names=cars_columns)
print(cars.shape)
cars.head()

(205, 26)


,symboling,normalized-losses,make,fuel-type,aspiration,number-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,number-of-cylinders,engine-size,fuel-system,bore,stroke,compression,horespower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


### 2) Pretend that this dataset represents the cars at a used car lot in your local town. 

- Use df.sample() to pick a random sample of 10 cars. Note that because this sample is random we are going to set the `random_state` so that all of us in the class get the same random sample. Please set your random state to `30` when using `df.sample()`

Is your sample reflective of the population value in regards to highway-mpg? Find the mean for `highway-mpg` for the entire dataset and compare your sample mean estimate to that value. Can you say that they are different? 

In [17]:
### YOUR WORK HERE
#finding the mean for the car data set
cars_mean = cars['highway-mpg'].mean()
car_sample = cars.sample(10, random_state=30)
car_sample_mean = car_sample['highway-mpg'].mean()
print(cars_mean)
car_sample_mean

30.75121951219512


32.8

The salesman says the cars he sells typically have a fuel efficiency of about 35 miles per gallon on the highway. You want to verify his claim but can only test 10 cars. Using your sample of 10, test his claim and report your results.

1) Null Hypothesis

$H_0: \overline{x} == \mu$

The salesman is telling the truth about milage

2) Alternative Hypothesis

$H_a: \overline{x} \neq \mu$

The salesman is lying about milage

3) Confidence level: 95%

In [19]:
### YOUR WORK HERE
stats.ttest_1samp(car_sample['highway-mpg'], 35)

Ttest_1sampResult(statistic=-0.9570244044334747, pvalue=0.36355472283248624)

4) Conclusion

Based on a t-statistic of -0.96, and a p-value of 0.36, I fail to reject the null hypothesis that the salesman is telling the truth.

Imagine that you now have the capacity to test 100 cars on the lot. Using the same random state of `30`, what would you conclude when sampling 100 cars?

1) Null Hypothesis

$H_0: \overline{x} == \mu$

The salesman is telling the truth about milage

2) Alternate Hypothesis

$H_a: \overline{x} \neq \mu$

The salesman is lying about milage

3) Confidence level: 95%

In [20]:
### YOUR WORK HERE
#creating new car sample
car_sample_100 = cars.sample(100, random_state=30)
stats.ttest_1samp(car_sample_100['highway-mpg'], 35)

Ttest_1sampResult(statistic=-6.5185242467032305, pvalue=3.003137417468754e-09)

4) Conclusion

Based on a t-statistic of -6.52, and a p-value of 0.000000003, I reject the null hypothesis that the salesman is telling the truth and suggest the alternative that he is lying

Why might these two t-tests using the same dataset lead to different conclusions about the salesman's claim?

The two t-test result in different conclusions due to the size of the sample. The first test only accounted for about 5% of the data, where the second accounted for about 50%.

## Practice 2-Sample T-tests

Two Sample t-tests determine whether or not two sample means are statistically different from each other. 

This portion of your assignment requires you to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). 


### 1) Load the data

The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

- Read the dataset in from UCI, you'll need to provide a list of column headers
- Encode "yes" votes as 1 and "no" votes as 0. (You can use `df.replace()` to swap out these values)
- Use dataframe filtering to split the dataframe into two new dataframes based on party. Hold all republicans in one dataframe and all democrats in the other. These will be our two different "samples."



In [21]:
### YOUR WORK HERE
#obtaining voting data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
!head house-votes-84.data

--2020-07-13 20:09:59--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2020-07-13 20:09:59 (497 KB/s) - ‘house-votes-84.data’ saved [18171/18171]

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [22]:
#creating dataframe
vote_names = ['party', 'handicapped-infants', 'water-project', 'budget', 'physician-fee-freeze', 
              'el-salvador-aid', 'school-religious-groups', 'anti-satellite-test-ban', 
              'nicaraguan-contra-aid', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 
              'education-spending', 'right-to-sue', 'crime', 'duty-free-exports', 'south-africa-export']
votes = pd.read_csv('house-votes-84.data', names=vote_names, na_values='?')
votes.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,school-religious-groups,anti-satellite-test-ban,nicaraguan-contra-aid,mx-missile,immigration,synfuels-corporation-cutback,education-spending,right-to-sue,crime,duty-free-exports,south-africa-export
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [24]:
votes = votes.replace({'y':1, 'n':0})
votes.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,school-religious-groups,anti-satellite-test-ban,nicaraguan-contra-aid,mx-missile,immigration,synfuels-corporation-cutback,education-spending,right-to-sue,crime,duty-free-exports,south-africa-export
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [25]:
#subsetting votes dataframe
r = votes[votes['party'] == 'republican']
d = votes[votes['party'] == 'democrat']

### 2) Perform two sample T-tests on different issues and report the results.

- Find an issue that democrats support more than republicans with p < 0.01 (significant at the 99% level).
- Find an issue that republicans support more than democrats with p < 0.01 (significant at the 99% level).
- Find an issue where the difference between republicans and democrats has p > 0.1 (Not significant at the 90% level - i.e. there may not be much of a difference the two sample means)

Please for each test that you run state your null and alternative hypothesis and then write a conclusion reflecting on the null and alternative hypothesis.

Remember, that two-sample t-tests will only tell us if the two groups are *different* from one another. We'll have to look at their sample means directly or use the sign on the t-statistic to know which of the two sample means is larger. 

#Do Democrats vote for the budget bill more often than Republicans?

1) Null Hypothesis

Democrats and Republicans support the bill at the same level

$H_0: \overline{x}_d == \overline{x}_r$

2) Alternate Hypothesis

Democrats and Republicans support the bill at different levels

$H_a: \overline{x}_d \neq \overline{x}_r$

3) Confidence Level: 99%

In [27]:
### YOUR WORK HERE
stats.ttest_ind(d['budget'], r['budget'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

4) Conclusion

Based on a t-statistic of 23.2 and a p-value of ~0, I reject the null hypothesis that both parties support the budget bill at an equal level and suggest the alternative that there are differing levels of support for the bill.

##Do Republicans vote for the education spending bill more often than Democrats?
1) Null Hypothesis

Republicans and Democrats support the bill at the same level

$H_0: \overline{x}_r == \overline{x}_d$

2) Alternate Hypothesis

Republicans and Democrats show a differing level of support for the bill

$H_a: \overline{x}_r \neq \overline{x}_d$

3) Confidence Level: 99%

In [29]:
stats.ttest_ind(r['education-spending'], d['education-spending'], nan_policy='omit')

Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

4) Conclusion

Based on a t-statistic of 20.5, and a p-value of ~0, I reject the null hypothesis that both parties suppor the bill at an equal level and suggest the alternative that there are differing levels of support for the bill.

## Stretch Goals:

### 1) Use functions and some iterator (for loop, .apply(), list comprehension, etc.) to perform two sample t-tests on every issue in the dataset in an automated fashion.

In [33]:
### YOUR WORK HERE
columns = ['handicapped-infants', 'water-project', 'budget', 'physician-fee-freeze', 
          'el-salvador-aid', 'school-religious-groups', 'anti-satellite-test-ban', 
          'nicaraguan-contra-aid', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 
          'education-spending', 'right-to-sue', 'crime', 'duty-free-exports', 'south-africa-export']
for column in columns:
  print(stats.ttest_ind(r[column], d[column], nan_policy='omit'))

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)
Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)
Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)
Ttest_indResu



### 2) Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### Be sure to check your work using Scipy!





In [44]:
### YOUR WORK HERE
#finding the upper half of the t-score
dt = d['school-religious-groups'].dropna()
tvotes = votes['school-religious-groups'].dropna()
tu = dt.mean() - .4
tu

0.07674418604651162

In [49]:
#finding the lower half of the t-score
from statistics import stdev
dsdt = stdev(dt)
srdt = np.sqrt(dt.value_counts().sum())
tl = dsdt / srdt
tl

0.031155388583850824

In [50]:
#calculate t-score
t = tu / tl
t

2.463271669360318

In [47]:
stats.ttest_1samp(d['school-religious-groups'], .4, nan_policy='omit')

Ttest_1sampResult(statistic=2.463271669360318, pvalue=0.014423519970770174)

### 3) Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Be sure to check your work using Scipy!

In [96]:
### YOUR WORK HERE
dt2 = d['immigration'].dropna()
rt2 = r['immigration'].dropna()

tu2 = rt2.mean() - dt2.mean()

dt2n = dt2.value_counts().sum()
rt2n = rt2.value_counts().sum()
dt2sd = stdev(dt2)
rt2sd = stdev(rt2)

tl2 = np.sqrt(((dt2sd ** 2) / dt2n + ((rt2sd ** 2) / rt2n)))

t2 = tu2 / tl2
t2

1.737470365966673

In [97]:
stats.ttest_ind(r['immigration'], d['immigration'], nan_policy='omit')

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [85]:
print(np.var(dt2))
np.var(rt2)

0.24918677442206771


0.24668503213957746

In [86]:
print(dt2n)
rt2n

263


165

In [87]:
print(dt2sd)
rt2sd

0.49918611200840485


0.49667396966176663

In [88]:
print(dt2.sum())
rt2.sum()

124.0


92.0

In [90]:
print(124 / 263)
print(dt2.mean())
print(92 / 165)
rt2.mean()

0.4714828897338403
0.4714828897338403
0.5575757575757576


0.5575757575757576

In [91]:
print(dt2.value_counts())
rt2.value_counts()

0.0    139
1.0    124
Name: immigration, dtype: int64


1.0    92
0.0    73
Name: immigration, dtype: int64

In [93]:
print(np.std(dt2))
np.std(rt2)

0.49918611200840485


0.49667396966176663

In [95]:
tu2

-0.08609286784191733